In [1]:
import os, time, json, random, numpy as np, pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
import faiss

# Load API key
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

print("✅ OpenAI client ready.")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


✅ OpenAI client ready.


In [2]:
df = pd.read_csv("/Users/pulinkumar/Desktop/ALGOVERSE/Financial-QA-10k.csv")
df = df.fillna("")
print(f"✅ Dataset loaded: {df.shape}")
print(df.head(3))


✅ Dataset loaded: (7000, 5)
                                            question  \
0  What area did NVIDIA initially focus on before...   
1  What are some of the recent applications of GP...   
2  What significant invention did NVIDIA create i...   

                                              answer  \
0           NVIDIA initially focused on PC graphics.   
1  Recent applications of GPU-powered deep learni...   
2                   NVIDIA invented the GPU in 1999.   

                                             context ticker    filing  
0  Since our original focus on PC graphics, we ha...   NVDA  2023_10K  
1  Some of the most recent applications of GPU-po...   NVDA  2023_10K  
2  Our invention of the GPU in 1999 defined moder...   NVDA  2023_10K  


In [6]:
from sentence_transformers import SentenceTransformer
import faiss

embed_model = SentenceTransformer("all-MiniLM-L6-v2")
context_embeddings = embed_model.encode(df["context"].tolist(), show_progress_bar=True)
index = faiss.IndexFlatL2(context_embeddings.shape[1])
index.add(context_embeddings)

def retrieve_contexts(question, top_k=1):
    q_vec = embed_model.encode([question])
    D, I = index.search(q_vec, top_k)
    return [df.iloc[i]["context"] for i in I[0]]


Batches: 100%|██████████| 219/219 [07:45<00:00,  2.13s/it]


In [ ]:
#  Answer Generation — GPT-4o-mini
def generate_answer_llm(question, context=None, model="gpt-4o-mini"):
    """Use GPT-4o-mini to generate concise financial answers."""
    if context:
        prompt = f"""
You are a financial analyst AI assistant.
Using the following financial filing excerpt, answer the question accurately and concisely.

Context:
{context}

Question: {question}

If context lacks sufficient data, respond with "Not enough information in the provided text."
"""
    else:
        prompt = f"You are a financial analyst. Answer clearly and factually:\n\n{question}"

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=250
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print("⚠️ Error (generator):", e)
        return None

#  Quick Test — Generate Financial Answer (GPT-4o-mini)

sample_q = "What significant invention did NVIDIA create in 1999?"
sample_ctx = retrieve_contexts(sample_q, top_k=1)[0]

out = generate_answer_llm(sample_q, context=sample_ctx, model="gpt-4o-mini")
print(json.dumps({"question": sample_q, "answer": out}, indent=2))



{
  "question": "What significant invention did NVIDIA create in 1999?",
  "answer": "NVIDIA created the GPU (Graphics Processing Unit) in 1999."
}


In [ ]:
#Batch Generation — GPT-4o-mini Answers
from tqdm import tqdm
import time

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

INPUT_PATH  = "/Users/pulinkumar/Desktop/ALGOVERSE/Financial-QA-10k.csv"
OUTPUT_PATH = "/Users/pulinkumar/Desktop/ALGOVERSE/Financial-Agent-GPT4oMini-QA.csv"

BATCH_SIZE  = 50     # adjust to 100–200 if you want longer runs
START_INDEX = 0      # change to resume later
generated_answers = []

# Load input file
df_input = pd.read_csv(INPUT_PATH).fillna("")
df_slice = df_input.iloc[START_INDEX:START_INDEX + BATCH_SIZE].copy()

print(f"✅ Loaded {len(df_slice)} questions for batch generation.")

for i, row in tqdm(df_slice.iterrows(), total=len(df_slice)):
    q = row["question"]
    try:
        ctx = retrieve_contexts(q, top_k=1)[0]
        ans = generate_answer_llm(q, context=ctx, model="gpt-4o-mini")
    except Exception as e:
        print(f"⚠️ Error at row {i}: {e}")
        ans = None
    generated_answers.append(ans)
    time.sleep(2)  # small pause for rate-limit safety

# Attach predictions
df_slice["gpt4o_mini_pred"] = generated_answers

# Save checkpoint batch
output_file = OUTPUT_PATH.replace(".csv", f"_batch_{START_INDEX}.csv")
df_slice.to_csv(output_file, index=False)

print(f"\n💾 Saved batch results to {output_file}")
print(f"✅ Generated {len(df_slice)} predictions successfully.")


✅ Loaded 50 questions for batch generation.


100%|██████████| 50/50 [02:53<00:00,  3.46s/it]


💾 Saved batch results to /Users/pulinkumar/Desktop/ALGOVERSE/Financial-Agent-GPT4oMini-QA_batch_0.csv
✅ Generated 50 predictions successfully.


In [ ]:
#  GPT-5 Semantic Judge — Structured & Clean Output
import re, json, time

def llm_as_judge(question, true_answer, pred_answer, model="gpt-5"):
    """
    GPT-5 evaluates semantic quality between the gold and predicted answers.
    Returns (score, explanation). Enforces strict JSON output.
    """
    system_msg = {
        "role": "system",
        "content": (
            "You are an expert financial QA evaluator. "
            "You must respond ONLY with valid JSON following this schema:\n\n"
            "{\n"
            '  "score": float between 0 and 1,\n'
            '  "explanation": "short reason (<40 words)"\n'
            "}\n\n"
            "No extra text, headers, or commentary."
        ),
    }

    user_msg = {
        "role": "user",
        "content": (
            f"Question: {question}\n\n"
            f"True Answer: {true_answer}\n\n"
            f"Predicted Answer: {pred_answer}\n\n"
            "Evaluate correctness, completeness, and relevance. "
            "Assign one combined score from 0.0–1.0."
        ),
    }

    try:
        resp = client.chat.completions.create(
            model=model,
            messages=[system_msg, user_msg],
            max_completion_tokens=200
        )

        raw = resp.choices[0].message.content.strip()

        # Try direct JSON load
        try:
            result = json.loads(raw)
        except json.JSONDecodeError:
            # Extract JSON-like content if extra text leaked
            match = re.search(r"\{.*\}", raw, re.DOTALL)
            result = json.loads(match.group(0)) if match else {"score": 0.0, "explanation": raw[:80]}

        score = float(result.get("score", 0.0))
        reason = result.get("explanation", "").strip()
        return score, reason

    except Exception as e:
        print("⚠️ Judge error:", e)
        return 0.0, str(e)


In [24]:
from tqdm import tqdm
import pandas as pd
import numpy as np

save_path = "/Users/pulinkumar/Desktop/ALGOVERSE/Financial-Agent-GPT5-Judge-Results.csv"

scores, reasons, answers = [], [], []
gen_model = "gpt-4o-mini"
judge_model = "gpt-5"

for i, row in tqdm(df.head(50).iterrows(), total=50):
    q, true_a = row["question"], row["answer"]
    ctx = retrieve_contexts(q, top_k=1)[0]

    # --- Generate with GPT-4o-mini ---
    try:
        pred = generate_answer_llm(q, context=ctx, model=gen_model)
    except Exception as e:
        print(f"⚠️ Generation error: {e}")
        pred = ""

    # --- Judge with GPT-5 ---
    for attempt in range(3):
        s, r = llm_as_judge(q, true_a, pred, model=judge_model)
        if s is not None:
            break
        print("⏳ Retry due to transient error...")
        time.sleep(5)

    answers.append(pred)
    scores.append(s)
    reasons.append(r)

    # --- Save every 10 samples ---
    if (i + 1) % 10 == 0:
        pd.DataFrame({
            "question": df.head(i + 1)["question"].values,
            "true_answer": df.head(i + 1)["answer"].values,
            "pred_answer": answers,
            "score": scores,
            "reason": reasons,
        }).to_csv(save_path, index=False)
        print(f"💾 Saved checkpoint {i+1}")

# --- Final save ---
pd.DataFrame({
    "question": df.head(len(scores))["question"].values,
    "true_answer": df.head(len(scores))["answer"].values,
    "pred_answer": answers,
    "score": scores,
    "reason": reasons,
}).to_csv(save_path, index=False)

print(f"✅ Completed evaluation → {save_path}")
print(f"Average Judge Score: {np.nanmean(scores):.3f}")


  0%|          | 0/50 [00:00<?, ?it/s]

 20%|██        | 10/50 [01:01<03:44,  5.62s/it]

💾 Saved checkpoint 10


 40%|████      | 20/50 [01:48<02:10,  4.37s/it]

💾 Saved checkpoint 20


...

💾 Saved checkpoint 30


 80%|████████  | 40/50 [03:16<00:43,  4.39s/it]

💾 Saved checkpoint 40


100%|██████████| 50/50 [04:17<00:00,  5.15s/it]

💾 Saved checkpoint 50
✅ Completed evaluation → /Users/pulinkumar/Desktop/ALGOVERSE/Financial-Agent-GPT5-Judge-Results.csv
Average Judge Score: 0.220
